# Selenium CGV 영화 리뷰 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--diable-dev-shm-usage')

     |████████████████████████████████| 911kB 4.4MB/s 
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ Packages [95.3 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates In

## CGV 영화 리뷰 긁어오기

* 아이언맨: http://www.cgv.co.kr/movies/detail-view/?midx=38262#1
* 다크나이트: http://www.cgv.co.kr/movies/detail-view/?midx=76417#1
* url을 통해 리뷰 페이지 접근 불가
* 셀레니움으로 페이지 번호를 클릭하여 접근

In [18]:
import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException

def get_movie_reviews(url, page_num=10):

  wd = webdriver.Chrome('chromedriver',options=chrome_options)
  wd.get(url)

  writer_list=[]
  date_list=[]
  review_list=[]

  for page_no in range(1, page_num+1):
    try:
      page_ul = wd.find_element_by_id('paging_point')
      page_a = page_ul.find_element_by_link_text(str(page_no))
      page_a.click()
      time.sleep(1)

      writers = wd.find_elements_by_class_name('writer-name')
      writer_list += [writer.text for writer in writers]

      reviews = wd.find_elements_by_class_name('box-comment')  
      review_list += [review.text for review in reviews]

      dates = wd.find_elements_by_class_name('day')
      date_list += [date.text for date in dates]

      if page_no % 10 == 0:
        next_button = page_ul.find_element_by_class_name('btn-paging next')
        next_button.click()
        time.sleep(1)
    except NoSuchElementException:
      break
      
  movie_review_df = pd.DataFrame({'Writer': writer_list,
                                  'Review': review_list,
                                  'Date': date_list})
  
  return movie_review_df

In [19]:
url =  'http://www.cgv.co.kr/movies/detail-view/?midx=83327'
movie_review_df = get_movie_reviews(url, 12)
movie_review_df

,Writer,Review,Date
0,늘푸른바다,두배우의 카리스마 대결 장난아님,2020.08.27
1,le**ns,액션 하나로 승부 보려 한다,2020.08.27
2,ki**the1,진짜 개 별로... 왜 봤디? 이걸 ? ^^,2020.08.27
3,강병주님,나쁘지 않았다 남자가 봐도 배우들이 섹시하다,2020.08.27
4,채식늑대,뭐가 뭔지... 그냥 멋있긴 함,2020.08.27
5,새콤보기,두 주연배우님의 연기 액션은 좋았으나 스토리의 개연성은 좀 떨어지는 것 같았습니다....,2020.08.27
6,jw**pp1,어지럽지만 신선함이 어느정도는 있었어요,2020.08.27
7,람보조,액션이 장난 아님...,2020.08.27
8,my**vegod9,잔인했지만 연기력 대박 재밌었어요,2020.08.27
9,ch**1052,다만 악 잼있네요..굿,2020.08.27


## CGV 상영작 스크래핑

* http://www.cgv.co.kr/movies/

In [22]:
url = 'http://www.cgv.co.kr/movies/'

wd = webdriver.Chrome('chromedriver', options=chrome_options)
wd.get(url)

movie_chart = wd.find_element_by_class_name('sect-movie-chart')
contents = movie_chart.find_elements_by_class_name('box-contents')

for content in contents:
  link = content.find_element_by_tag_name('a').get_attribute('href')
  title = content.find_element_by_class_name('title').text
  percent = content.find_element_by_class_name('percent').text
  info = content.find_element_by_class_name('txt-info').text
  print(title, percent, info, link)
  print(get_movie_reviews(link, 2))

테넷 예매율88.7% 2020.08.26 개봉 http://www.cgv.co.kr/movies/detail-view/?midx=83381
       Writer                                             Review        Date
0    kw**oc25           다시 보고 싶은 영화. 놀란 감독의 영화는 두 번 관람이 필수가 되었다.  2020.08.27
1          쪼소  영화 자체는 굉장히 좋았어요. 타임 패러독스 영화를 본적 있는데 그 영화의 좀 더 ...  2020.08.27
2       십리도화림  1회차로는 이해하기 쉽지 않은 영화. 3회차 정도 하니 이해되지 않았던 것들, 보이...  2020.08.27
3      bp**04                         어려운 건 괜찮은데 재미가 없는 건 안 괜찮아.  2020.08.27
4    jh**nice     시공간의 개념을 철학적으로 보여주는 최고의 영화. 마지막 전투신은 신선한 충격이다.  2020.08.27
5     ja**oov              모든 시작엔 이유와 끝이 있으며 일련의 과정은 흐름대로 느끼면 된다  2020.08.27
6     pd**205                                    아이맥스로 봐야되요 재밌어요  2020.08.27
7        스트롱스                                       영상미가 좋은 영화네요  2020.08.27
8     pk**487                                       재미있고 기대이상입니다  2020.08.27
9     le**n11                                    어려웠다. 그냥 액션만 봤다  2020.08.27
10       황소바우                                   어렵네요. 한번 더 볼까봐요.  2020.08.2